# Local Delivery Planner for Amazon Regular Orders

A sample notebook for generating local delivery plans of Amazon Regular Orders

### Imports

In [ ]:
import json
import pandas as pd

from amznoptim.solver.GurobiMIP import DepotVRPGurobiRegular
from amznoptim.utils import preprocess

### Input preparation

In [ ]:
# Example order csv file
order_csv = "formatted_data/regular_orders.csv"
pd.read_csv(order_csv)

In [ ]:
# Example depot info file
depot_json = "formatted_data/depot.json"
with open(depot_json, "r") as f:
    depot_info = json.load(f)
depot_info

In [ ]:
# Example Google Map RouteMatrix json
gmap_json = "formatted_data/gmap_matrix.json"
# Example vehicle config json
vehicle_json = "formatted_data/vehicle.json"
# Example Google Map Address Validation json
validation_json = "formatted_data/gmap_validation.json"

### Solver data processing

In [ ]:
# Preprocess the order data csv and extract order and address info
order_info, address_info = preprocess.regular_order_info_from_csv(
    order_csv, "../sample_data/standard_box/boxsize.tsv"
)
order_info

### Use Gurobi MIP Solver (May require Gurobi WLS License File)

For licensing, please visit https://www.gurobi.com/features/web-license-service/

Alternatively, you can run Google OR-Tools CP-SAT solver below (slower than Gurobi)

In [ ]:


# Define the Gurobi solver and process the data
solver = DepotVRPGurobiRegular(
    order_data=order_info,
    address_data=address_info,
    depot_data=depot_info,
)
solver.process_data(
    vehicle_data_path=vehicle_json,
    matrix_json=gmap_json,
)

solver.set_stopping_time(validation_json=validation_json)

# Optional: use your own Gurobi WSL license file
# solver.use_licensed_environment('path/to/your/gurobi.lic')

#### Run the solver

In [ ]:
result = solver.solve()
result

#### Process the result to human-readable format

In [ ]:
result_gurobi_readable = solver.generate_plan(result, save_path=None)
result_gurobi_readable

### Alternative: Use CP-SAT solver (Deprecated)


In [ ]:
from amznoptim.solver.CpSAT import DepotVRPCpRegular

# Define the CP-SAT solver and process the data
solver = DepotVRPCpRegular(
    order_data=order_info,
    address_data=address_info,
    depot_data=depot_info,
)
solver.process_data(
    vehicle_data_path=vehicle_json,
    matrix_json=gmap_json,
)
solver.set_stopping_time(validation_json=validation_json)

#### Run the solver

In [ ]:
result = solver.solve()
result

#### Process the result to human-readable format

In [ ]:
result_readable = solver.generate_plan(result, save_path=None)
result_readable